# Analyzing IMDB Data in Keras

In [1]:
# Imports
import numpy as np
from keras_helpers import AlphaDropout, selu, lecun_normal
import keras
from keras.engine import Layer
from keras.datasets import imdb
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input
from keras.optimizers import SGD
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)

Using TensorFlow backend.


## 1. Loading the data
This dataset comes preloaded with Keras, so one simple command will get us training and testing data. There is a parameter for how many words we want to look at. We've set it at 1000, but feel free to experiment.

In [2]:
# Loading the data (it's preloaded in Keras)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=1000)

print(x_train.shape)
print(x_test.shape)

(25000,)
(25000,)


## 2. Examining the data
Notice that the data has been already pre-processed, where all the words have numbers, and the reviews come in as a vector with the words that the review contains. For example, if the word 'the' is the first one in our dictionary, and a review contains the word 'the', then there is a 1 in the corresponding vector.

The output comes as a vector of 1's and 0's, where 1 is a positive sentiment for the review, and 0 is negative.

In [3]:
print(x_train[0])
print(y_train[0])

[1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]
1


## 3. One-hot encoding the output
Here, we'll turn the input vectors into (0,1)-vectors. For example, if the pre-processed vector contains the number 14, then in the processed vector, the 14th entry will be 1.

In [4]:
# One-hot encoding the output into vector mode, each of length 1000
tokenizer = Tokenizer(num_words=1000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print(x_train[0])

[ 0.  1.  1.  0.  1.  1.  1.  1.  1.  1.  0.  0.  1.  1.  1.  1.  1.  1.
  1.  1.  0.  1.  1.  0.  0.  1.  1.  0.  1.  0.  1.  0.  1.  1.  0.  1.
  1.  0.  1.  1.  0.  0.  0.  1.  0.  0.  1.  0.  1.  0.  1.  1.  1.  0.
  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  1.  1.  0.  0.  0.  0.  1.
  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  1.  0.
  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  1.  1.  0.  1.  1.
  0.  0.  0.  0.  1.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  1.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  1.  0.  0.
  1.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

And we'll also one-hot encode the output.

In [5]:
# One-hot encoding the output
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(25000, 2)
(25000, 2)


## 4. Building the  model architecture
Build a model here using sequential. Feel free to experiment with different layers and sizes! Also, experiment adding dropout to reduce overfitting.

In [6]:
# TODO: Build the model architecture
inputs = Input(shape=(1000,))
x = Dense(4096, kernel_initializer=lecun_normal(seed=42))(inputs)
x = Activation(selu)(x)
x = AlphaDropout(0.1)(x)
x = Dense(2048, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation(selu)(x)
x = AlphaDropout(0.1)(x)
x = Dense(1024, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation(selu)(x)
x = AlphaDropout(0.1)(x)
x = Dense(512, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation(selu)(x)
x = AlphaDropout(0.1)(x)
x = Dense(512, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation(selu)(x)
x = AlphaDropout(0.1)(x)
x = Dense(256, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation(selu)(x)
x = AlphaDropout(0.1)(x)
x = Dense(256, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation(selu)(x)
x = AlphaDropout(0.1)(x)
x = Dense(128, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation(selu)(x)
x = AlphaDropout(0.1)(x)
x = Dense(128, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation(selu)(x)
x = AlphaDropout(0.1)(x)
x = Dense(64, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation(selu)(x)
x = AlphaDropout(0.1)(x)
x = Dense(64, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation(selu)(x)
x = AlphaDropout(0.1)(x)
outputs = Dense(2, activation='softmax')(x)
model = Model(inputs, outputs)

# TODO: Compile the model using a loss function and an optimizer.
optim = SGD(decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=optim,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## 5. Training the model
Run the model here. Experiment with different batch_size, and number of epochs!

In [7]:
# TODO: Run the model. Feel free to experiment with different batch sizes and number of epochs.
early = EarlyStopping(patience=3)
model.fit(x_train, y_train, batch_size=32, epochs=150, validation_split=.2, verbose = 2, callbacks=[early])

Train on 20000 samples, validate on 5000 samples
Epoch 1/150
7s - loss: 0.5592 - acc: 0.7040 - val_loss: 0.4425 - val_acc: 0.8426
Epoch 2/150
5s - loss: 0.3906 - acc: 0.8319 - val_loss: 0.4203 - val_acc: 0.8552
Epoch 3/150
5s - loss: 0.3599 - acc: 0.8423 - val_loss: 0.4327 - val_acc: 0.8590
Epoch 4/150
5s - loss: 0.3407 - acc: 0.8545 - val_loss: 0.4490 - val_acc: 0.8526
Epoch 5/150
5s - loss: 0.3342 - acc: 0.8577 - val_loss: 0.4394 - val_acc: 0.8576
Epoch 6/150
5s - loss: 0.3270 - acc: 0.8596 - val_loss: 0.4568 - val_acc: 0.8572


## 6. Evaluating the model
This will give you the accuracy of the model, as evaluated on the testing set. Can you get something over 85%?

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: ", score[1])

Accuracy:  0.85648
